In [ ]:
from video_commands import *
main_video = "B0815_subtitle.mov"
width,height = get_media_dimensions(main_video)
print(width,height)

## check dimensions of a file

In [ ]:
width,height = get_media_dimensions("black.mov")
print(width,height)



## trim Video
outvideo = "test.mov"<br>
invideo = "B0815.mov"<br>
starttime,endtime = "0:10" , "0:18"<br>

In [ ]:
from video_commands import *
outvideo = "test_5min.mov"
invideo = "B0815_2flower.mov"
starttime,endtime = "17:10" , "20:10"
trim_video(invideo, "16:30" , "21:30", outvideo)
"""
trim_video("LETTER.mp4", "00:00" , "00:15", "letter_tmp.mov")
trim_video("黄花菜地.mov", "00:02" , "00:22", "黄花菜地_tmp.mov")
trim_video("beginning_ban.mov", "00:00" , "00:10", "beginning_ban_tmp.mov")
x, muted_video1, z = separate_audio_video("letter_tmp.mov",".",0)
x, muted_video2, z =separate_audio_video("黄花菜地_tmp.mov",".",0)
x, muted_video3, z =separate_audio_video("beginning_ban_tmp.mov",".",0)
a_list = [muted_video1, muted_video2, muted_video3]
a,b=reencode_to_match(outvideo,a_list,crf="30",preset="veryfast")
print(a)
print_media_info(a)
"""

## add a black still and insert in video<br>
input_video = "test_1min.mov"<br>
outfile = "test.mov"<br>
insert_black_stills(input_video=input_video,black_list=black_list,output_file=outfile,crf=18,preset="fast")



# # ADD multiple black stills and insert into video
<font color = "red">add def, insert multiple new black stills using concat. Ask me to delete original mov</font>



In [ ]:
from video_commands import *
input_video = "test_5min.mov"
start_time = 0#"17:00.5" #5
end_time = 5#"17:03.5"   #10
crf = 10
preset = "fast"
input_file=[["28年后的巨变","black1.mov"],["日本留学","black2.mov"],["回国梦","black3.mov"]]
#for i in input_file:
#    create_black_still(input_video, i[0],start_time , end_time, i[1], font_name="Arial",font_size=130,font_color="&H00FFFFFF")

video_duration = get_video_length(input_video)
sections = ["0:00-1:00", "1:00-2:00", "2:00-3:00",f"3:00-{video_duration}"]
input_file = "test_5min.mov"
outputfiles = ["video1.mov","video2.mov","video3.mov","video4.mov"]
#for section, outfile in zip(sections, outputfiles):
#     split_video(input_file, [section], [outfile])
video_list =["video1.mov","black1.mov","video2.mov","black2.mov","video3.mov","black3.mov","video4.mov"]
#combine_video(video_list,primary_video_index= ,output_file="test.mov", crf=40, preset="ultrafast")
combine_video(video_list,0,output_file="test_final3.mov", crf=18, preset="fast")

In [ ]:
from video_commands import * 
success, temp_muted, audio_file = separate_audio_video("war_muted.mov",".",0)
main_video = "test_3min.mov"
list_to_reencode = ['war_muted.mov']
a,b = reencode_to_match(main_video, list_to_reencode,crf="18",preset="fast")
print(a) # names of reencoded videos
print(b) #dictionary with original and reencoded name
a = ["tmp/test_3min_reencoded_padded.mov","tmp/rescaled_clip_reencoded_padded.mov"]
print_media_info(a)
a = get_video_length("test_3min.mov")
print(a)


## Audio - converting mp3 to wav<br>
mp3_file = "music/darkness.mp3"

In [ ]:
mp3_file = "music/darkness.mp3"
mp3_to_wav(mp3_file)

## Audio - adjusting volume<br>
wav_file = "music/darkness.wav" <br>
volume_factor = 0.1

In [ ]:
wav_file = "music/darkness.wav"
volume_factor = 0.1
adjust_wav_volume(wav_file, volume_factor)

## split video into sections<br>
sections = ["0:00-5:00", "5:00-8:00", "9:00-12:00"]<br>
split_video("xxx.mov", sections)<br>
output:test_5min_0005-0100.mov,test_5min_0100-0200.mov,test_5min_0220-0300.mov


In [ ]:
sections = ["0:05-1:00", "1:00-2:00", "2:20-3:00"]
input_file = "test_5min.mov"
split_video(input_file, sections)

## extract voice from a video and convert to srt file
lang can be None


In [ ]:
input_video, output_srt,lang = "test_1min.mov", "test_1min.srt","zh"
voice_to_srt(input_video, output_srt, lang=lang)#en,ja,k

## srt to ass

In [ ]:
srt_file = "test_1min.srt" #input file
ass_file = "test_1min.ass" #new file
srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30)
burn_subtitles("test_1min.mov", "test_1min.ass", "output.mov", "Arial")


## Translation and Combine Different Translations for proofreading
<h3>only for srt!!!</h3>

In [ ]:
inputfile,outputfile,lang = "test_1min.srt","test_1min_jp.srt","JA" #lang = target language
deepl_translate_srt(inputfile, outputfile ,lang=lang)#EN-US KO zh JA

In [ ]:
inputfile = "test_1min.srt"
outputfile = "test_1min_jp.srt"
combined_file = "combined_chi_jp.srt"  #new file, oneline chinese one line japanese
combine_srt(inputfile,outputfile, combined_file)

## Add grid to a screenshot for mosaic
add_numbered_grid(<br>
    input_file,output_file,width,height,interval=100,line_color="red",<br>
    number_color="yellow",number_size=30,number_offset=5)

In [ ]:
time_and_name = [
        ["1:10", "test1.png"],           # 1 minute 10 seconds
        ["70", "test2.jpg"],             # 70 seconds
        ["1:30.5", "test3.png"],         # 1 minute 30.5 seconds
        ["00:01:45", "test4.jpg"],       # 1 minute 45 seconds (full format)
        ["00:02:30.750", "test5.png"],   # 2 minutes 30.75 seconds
        ["4:41", "test6.jpg"]            # 4 minutes 41 seconds
        ]
extract_frames("test_5min.mov", time_and_name)

In [ ]:
for i in time_and_name:
    outscreenshot = f"{i[1].split(".")[0]+"_grid.png"}"
    add_numbered_grid(
    i[1],
    outscreenshot,
    width,
    height,
    interval=50,
    line_color="red",
    number_color="yellow",
    number_size=30,
    number_offset=5
)

# Apply Mosaic

## example: <br>
mosaic_list=[<br>
        [50, 50, 100, 50, 10, 0, 10],      <br>
        [800, 0, 200, 200, 15, "1:20.5", "3:00"]]  <br>
    # position of top-left corner<br>
    x=50,y=50,width=100,height=50<br>
    pixelization level=10<br>
    start seconds=0,end seconds=10<br>

In [ ]:
input_video,output_video = "test_5min.mov","test_5min_mosaic.mov"
mosaic_list = [[65, 50, 200, 170, 15, 0, 30],[width-260, 60, 285, 140, 15, "1:20.5", "2:30"]]
apply_mosaics(input_video, output_video, mosaic_list)

## srt to ass and burn in subtitles <br>
srt_file = "../中文字幕.srt" #input file <br>
ass_file = "中文字幕.ass" #new file <br>
input_video = "B0815_4letter.mov" <br>
output_video = "B0815_subtitle.mov" <br>
#srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30) <br>
burn_subtitles(input_video, ass_file, output_video, "Arial",crf ="18",preset="veryfast") <br>

In [ ]:
srt_file = "中文字幕.srt" #input file
ass_file = "中文字幕.ass" #new file
input_video = "B0815_4letter.mov"
output_video = "B0815_subtitle.mov"
srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30)
burn_subtitles(input_video, ass_file, output_video, "Arial",crf ="10",preset="fast")

In [ ]:
srt_file = "test_1min.srt" #input file
ass_file = "test_1min.ass" #new file
input_video = "test_1min.mov"
output_video = "test.mov"
srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30)
burn_subtitles(input_video, ass_file, output_video, "Arial")

## End film
input "end_subtitle_txt" in the end film this way:<br>
编导演播：翰 光<br>
音乐：Chad Cannon<br>
照片提供： 小林熏<br>
         李晓斌<br>

The content of <b>end_subtitle_txt</b> is inserted into <b>end_subtitle_template</b> <br>
as <b>end_subtitle_ass</b> by <b><i>end_credits_ass</b></i>.<br>
Then <b><i>create_ending_film</b></i> is called to create <b>end_output</b> with <b>end_song</b> as background music.<br>
The dimension of  <b>end_output</b> is the same as the <b>main_video</b>. The background music has an adjustable volume<br>
and fade-in,fade-out time. <b>end_second</b> is the silent time before the end of the film and after the end of the song.
You can add a background video with an adjustable fade-out time. <br>
Remember to use <b><i>preprocess</b></i> to rescale it first.<br>
Add rescale to match the scale

In [ ]:
end_subtitle_ass= "2508credits.ass"
end_subtitle_txt= "end_subtitle.txt"
end_subtitle_template = "end_subtitle_template.ass"
end_song = "music/end_song.wav"
end_output3 = "end_vol3.mov"
end_credits_ass(end_subtitle_template, end_subtitle_txt, end_subtitle_ass,first_start="0:00:01.40", 
                 each_duration_s=6.0,line_offset_s=0.60, fallback_duration_s=4.0)
create_ending_film(end_subtitle_ass, end_song, end_output3, width, height,
                       volume=3, end_second=5, fadein=3, fadeout=10,
                       bg_video=None,video_fadeout=4)

## Reencode and combine 
Do the reencoding inside
output_file="B0815_final1.5.mp4" not ok,must be <br>
output_file="B0815_final1.mp4", no "." in between

In [ ]:
from video_commands import *
#main_video = "test_3min.mov"
#width,height = get_media_dimensions(main_video)
list_to_reencode = ["beginning_ban.mp4","end_vol3.mov"]
video_list = ["beginning_ban.mov","test_final3.mov","end_vol3.mov"]
#make_rescaled_image(width, height, "beginning_ban.mp4", "beginning_ban_rescaled.mp4")
#make_rescaled_image(width, height, "end_vol3.mov", "end_vol3_rescaled.mov4")
#a,b= reencode_to_match2(main_video, list_to_reencode)
#print(a)
#print(b)
#files = ["tmp/1_test_3min_reencoded.mov","tmp/0_beginning_ban_reencoded.mov","tmp/2_end_vol3_reencoded.mov"]
#print_media_info(files)

combine_video(video_list, primary_index=1,output_file="B0815_final_3min.mov", crf=18,preset="fast")


In [ ]:
#complicated method
time1,time2 = "13:09.8","13:14.9"
duration = parse_time(time2)-parse_time(time1)
print(duration)
overlay_video_split_and_combine(
    main_video="B0815_3flower.mov",
    overlay_video="video_folder/letter_muted.mp4",
    overlay_start="0:02",         # trim overlay from 0:02-0:02+duration
    overlay_duration = duration,        # modify to the end
    main_insert_at= time1,    
    output_file="B0815_4letter.mov",# insert overlay at 5:00 of main
    crf=10,
    preset="fast",
    keep_intermediates=True
)



In [ ]:
make_rescaled_image(width, height, "班和保人.png", "test.png")

In [ ]:
from video_commands import *
main_video = "test_3min.mov"
output_file = "overlay_test2.mov"
para_list = [["letter.MP4","1:30","1:45","0:00","2"],["war.mov","2:00","2:20","0:02","0"],["taiwan.MP4","2:40","2:50","0:01","5"]]
overlay_video(main_video,para_list,output_file,crf="23",preset="fast")

1.insert multiple images to overlay_video<br>
2.overlay a single image, or list image<br>
3.overlay_black_still not working in github<br>
4. 更改片尾字幕resolution<br>
5.overlay_music<br>
6. use the following to check real video size to adjust rescaling for vertical videos<br>
ffmpeg -i video.mp4 -vf "cropdetect=24:16:0" -frames:v 1 -f null -<br>
or<br>
import cv2<br>
import numpy as np<br>
<br>
cap = cv2.VideoCapture("video.mp4")<br>
ret, frame = cap.read()<br>
if ret:<br>
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)<br>
    # Count non-black pixels<br>
    non_black = np.sum(gray > 16)<br>
    total_pixels = gray.size<br>
    # Simple ratio of non-black region<br>
    ratio = non_black / total_pixels<br>
    print("Non-black ratio:", ratio)<br>
cap.release()<br>



import cv2
import numpy as np

cap = cv2.VideoCapture("xingxing.mp4")
ret, frame = cap.read()
if ret:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Count non-black pixels
non_black = np.sum(gray > 16)
total_pixels = gray.size
# Simple ratio of non-black region
ratio = non_black / total_pixels
print("Non-black ratio:", ratio)
cap.release()

<h3> ***continue here , burn in subtitle </h3>

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture("xingxing.mp4")
ret, frame = cap.read()
if ret:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Create a mask of non-black pixels
    mask = gray > 16  # threshold to consider pixel as "non-black"
    
    # Find coordinates of non-black pixels
    coords = np.column_stack(np.where(mask))
    
    # Get bounding rectangle
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    
    active_width = x_max - x_min + 1
    active_height = y_max - y_min + 1
    
    print("Active width:", active_width)
    print("Active height:", active_height)
    
    if active_height > active_width:
        print("Orientation: Portrait")
    else:
        print("Orientation: Landscape")
else:
    print("Could not read frame")

cap.release()


## notes
reencode_to_match()<br>
reencode videos in aac, but supposed to be in pcm